In [36]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
import scipy, scipy.signal
from scipy.io import loadmat
import os
import pandas as pd
import datetime as dt
 
# import constants
c = 2.998e8 # m/s 
D2R = np.pi/180.
R2D = 180./np.pi
Hz2Rad = 2.*np.pi
Rad2Hz = 1./Hz2Rad
Q_EL = 1.602e-19
M_EL = 9.1e-31
eo   = 8.854e-12
B0   = 30e-6
mu   = 4.0*np.pi*10e-7

In [42]:
# import input data from VLF on table mtn
datadir = 'vlf_data'
datafiles = [f for f in os.listdir(datadir) if os.path.isfile(os.path.join(datadir, f))]

# x data is the 000.mat file 
for dfile in datafiles:
    if dfile[-5] == '0':
        bx_datafile = os.path.join(datadir, dfile)
    else:
        by_datafile = os.path.join(datadir, dfile)

# load mat files
bx_data = loadmat(bx_datafile)
by_data = loadmat(by_datafile)

# grab start time
data_start = dt.datetime(int(bx_data['start_year']), int(bx_data['start_month']), int(bx_data['start_day']), int(bx_data['start_hour']), int(bx_data['start_minute']), int(bx_data['start_second']))

# get data -- 10 min of data? 
bx_data = bx_data['data']
by_data = by_data['data']

# create a timevec for the data
data_dt = 100e3 / len(bx_data)
data_dt = dt.timedelta(microseconds=data_dt*1e6) # convert time int in data to microseconds
data_dt_vec  = [data_start+(data_dt*(i+1)) for i, i_data in enumerate(bx_data)]

# convert to a dataframe
data_dt_vec = pd.to_datetime(data_dt_vec)
d_bx = {'time': data_dt_vec, 'data': bx_data}
d_by = {'time': data_dt_vec, 'data': by_data}

df_bx = pd.DataFrame(d_bx)
df_by = pd.DataFrame(d_by)

# data is 100kHz, but canvas sampling is 2^17 (131072 Hz) -- resample data
# use pandas resampling func to resample an interpolate - using polynomial order 2
upsampled = series.resample('D')
interpolated = upsampled.interpolate(method='spline', order=2)

# finally, convert to 16 bit int for input time series data
bx_array = np.array()
by_array = np.array()

bx_list = [np.int16(bxi) for bxi in bx_data]
bx_list = [np.int16(bxi) for bxi in bx_data]

bx = np.array(bx_list)
by = np.array(by_list)
# ---------------- checkpoint -------------

TypeError: to_datetime() missing 1 required positional argument: 'arg'

In [ ]:
# 2048-point FFTs with 1024-point overlap, and accumulate for 1 second
window = 'hanning'
nfft = 2048
overlap = 0.5

# for a spectra
ff, tt, Fbx = scipy.signal.spectrogram(bx, fs=fs, window=window, nperseg=nfft, noverlap=nfft*overlap,mode='complex')
# should mode be complex only for cross spectra? 

# output will be 2048 points covering frequencies from 0 to fs/2 and -fs/2 to 0
print(np.int32(FEx)) # check FFT output

# energy is multiplied by 2 (for the negative half)

In [ ]:
S_mag = np.sqrt(np.real(FEx*np.conj(FEx))) # dont need to take the real part here
# no square root - squared magnitude [V^2/(m^2 Hz)]
# check spectrogram function mode = complex (output units)
# then multiply by 2 
# output check 

In [ ]:

fname = 'fbins.txt'
fbins = np.genfromtxt(fname, dtype='str')
# bin and average -- bin same as the firmware will 
# accumulate 128 here and average 